In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path
from itertools import chain
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
import joblib

# Mis utilitarios
from utils import *


In [17]:

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Concatenate, ConvLSTM2D
from tensorflow.keras.initializers import Constant
from tensorflow.keras.utils import plot_model

# Set Path

In [4]:
el_path_main = Path(os.getcwd()).parent
el_path_main

WindowsPath('e:/backup Asus G15 27_10_2021/Colegio de Mates Bourbaki/DLA2501 - Deep Learning Avanzado/Flood forecasting')

In [35]:
# En caso sea en google Colab

# Load Tensors

In [5]:
X_gts_train, Y_train, X_gts_miss_train, X_ctry_lvl_train = joblib.load(el_path_main / '2. Datos/Processed data/TRAIN_data.pkl')
X_gts_oot, Y_oot, X_gts_miss_oot, X_ctry_lvl_oot = joblib.load(el_path_main / '2. Datos/Processed data/OOT_data.pkl')

In [ ]:
# En caso sea en google Colab
X_gts_train, Y_train, X_gts_miss_train, X_ctry_lvl_train = joblib.load(el_path_main / '2. Datos/Processed data/TRAIN_data.pkl')
X_gts_oot, Y_oot, X_gts_miss_oot, X_ctry_lvl_oot = joblib.load(el_path_main / '2. Datos/Processed data/OOT_data.pkl')

In [6]:
X_gts_oot.shape, Y_oot.shape, X_gts_miss_oot.shape, X_ctry_lvl_oot.shape

((193, 10, 96, 36, 28), (193, 12, 36, 28), (193, 10, 96, 36, 28), (193, 180))

In [7]:
X_gts_train.shape, Y_train.shape, X_gts_miss_train.shape, X_ctry_lvl_train.shape

((1177, 10, 96, 36, 28),
 (1177, 12, 36, 28),
 (1177, 10, 96, 36, 28),
 (1177, 180))

# Conv-LSTM

In [24]:
n_vars_from_gts = X_gts_train.shape[1] # N of variables from GTS
n_times_for_lstm = X_gts_train.shape[2] # N of time steps usted for LSTM
n_lats = X_gts_train.shape[3] # N of latitudes
n_lons = X_gts_train.shape[4] # N of longitudes

# Set of parameters that will be tunned
recurrent_dropout = 0.2
dropout = 0.2
n_filters = 2

In [25]:
X_prueba = X_gts_train[:,0,:]
X_prueba.shape

(1177, 96, 36, 28)

In [ ]:

inp1 = Input(shape=(n_times_for_lstm, n_lats, n_lons, 2, ), name='inp1')

mi_lstm= ConvLSTM2D(
                    filters = n_filters,
                    kernel_size = (3, 3), # Tamaño del kernel
                    data_format='channels_last', # Forma en que recibe los datos:Docu -> 5D tensor with shape: (samples, time, rows, cols, channels)
                    return_sequences=True, # Retorna cada secuencia de salida
                    recurrent_dropout=recurrent_dropout, dropout=dropout  # Para evitar sobreajuste
                    )(inp1)

mi_out = Dense(1, activation='linear', name='mi_out')(mi_lstm)


CODE_model = Model(inputs=[inp1], outputs = mi_out)  # unión del modelo
CODE_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ inp1 (InputLayer)               │ (None, 96, 36, 28, 2)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_9 (ConvLSTM2D)      │ (None, 96, 34, 26, 2)  │           296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mi_out (Dense)                  │ (None, 96, 34, 26, 1)  │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 299 (1.17 KB)

 Trainable params: 299 (1.17 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

# Input gts
dict_gts_inputs = {}
dict_gts_lstms = {}
for i in range(n_vars_from_gts):
    dict_gts_inputs[f'input_gts_{i}'] = Input(shape=(1,), name=f'input_gts_{i}')

    dict_gts_lstms[f'lstm_gts_{i}'] = LSTM(units = hidden_units_lstm,
                              return_sequences=False, # No retornamos todos los estados h para cada timestep (caracter) (en Name entity recog le pusimos True para scar los predicts de cada palabra)
                              recurrent_dropout=0.2, dropout=0.2  # Para evitar sobreajuste
                              )(dict_gts_inputs[f'input_gts_{i}'])



# Benchmark (Logit)